In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import pickle
from sklearn.metrics import classification_report, precision_score, recall_score, f1_score, accuracy_score, \
                            confusion_matrix, plot_confusion_matrix, precision_recall_curve, plot_roc_curve, \
                            roc_auc_score
from sklearn.model_selection import train_test_split, cross_val_score

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer()

In [3]:
def classifier_report(y_true, y_pred):
    
    print("Precision :  ", precision_score(y_true, y_pred))
    print("Recall    :  ", recall_score(y_true, y_pred))
    print("F1-score  :  ", f1_score(y_true, y_pred))
    print("Accuracy  :  ", accuracy_score(y_true, y_pred))
    print("ROC AUC   :  ", roc_auc_score(y_true, y_pred))
    print("Confusion Matrix : ")
    print(confusion_matrix(y_true, y_pred))

In [4]:
from sklearn.ensemble import AdaBoostClassifier
adaboost = AdaBoostClassifier()

In [5]:
data = pd.read_csv("../preprocessed_data.csv")
test_data = pd.read_csv("../preprocessed_test_data.csv").drop(columns=["id"])
# data, test_data = train_test_split(data, test_size=0.3, random_state=42)

In [6]:
def clean(tweet):
    
    new_tweet = ""
    for word in tweet.split():
        if word not in most_common:
            new_tweet += word
            new_tweet += " "
        
    return new_tweet

most_common = []
for word in Counter("".join(data["tweet"]).split()).most_common()[:20]:
    most_common.append(word[0])

data["tweet"] = data["tweet"].apply(clean)
test_data["tweet"] = test_data["tweet"].apply(clean)

In [7]:
np.array(cross_val_score(adaboost, data.drop(columns=["tweet", "label"]), data["label"], cv=10)).mean()

0.8905303030303031

In [8]:
adaboost.fit(data.drop(columns=["tweet", "label"]), data["label"])

AdaBoostClassifier()

In [9]:
prediction = pd.DataFrame({"label": adaboost.predict(test_data.drop(columns=["tweet"]))})
prediction.to_csv("adaboost.csv")

In [10]:
pickle.dump(adaboost, open("adaboost.sav", "wb"))

In [11]:
prediction["label"].value_counts(normalize=True)

0    0.710189
1    0.289811
Name: label, dtype: float64